In [4]:
self.mdft.xc = "b3lyp"
dm1 = self.mdft.make_rdm1()
ao_value = ni.eval_ao(self.mol, self.grids.coords, deriv=1)
rho = ni.eval_rho(self.mol, ao_value, dm1, xctype="MGGA")
exc, vxc = dft.libxc.eval_xc(self.mdft.xc, rho)[:2]
vxc = vxc[0]
xc_v = self.aux_function.oe_fock(vxc, self.grids.weights)
# exc +=
self.logger.info(f"\nExc = {np.sum(exc * rho[0] * self.grids.weights)}\n")

e_nuc = oe.contract("ij,ji->", self.h1e, dm1)
e_vj = oe.contract("pqrs,pq,rs->", self.eri, dm1, dm1)
e_vk = oe.contract("pqrs,pr,qs->", self.eri, dm1, dm1)
ene_t_vc = (
    e_nuc
    + self.mol.energy_nuc()
    + e_vj * 0.5
    - e_vk * 0.05
    + (exc * rho[0] * self.grids.weights).sum()
)
self.logger.info(f"Ene_t_vc = {ene_t_vc}\n")
self.logger.info(f"Total energy = {self.mdft.e_tot}\n")
self.logger.info(f"Shape of vxc: {np.shape(vxc)}\n")
self.logger.info(f"Shape of weights: {np.shape(self.grids.weights)}\n")
self.logger.info(
    f"Before scf: {np.array2string(dm1, precision=4, separator=',', suppress_small=True)}\n"
)

for step in range(200):
    rho = ni.eval_rho(self.mol, ao_value, dm1, xctype="MGGA")
    exc, vxc = dft.libxc.eval_xc(self.mdft.xc, rho)[:2]
    rho_1 = 2 * np.einsum("uv, rgu, gv -> rg", dm1, self.ao_1, self.ao_0)

    xc_v = 0.5 * self.aux_function.oe_fock(vxc[0], self.grids.weights)
    xc_v += 2 * oe.contract(
        "g, g, rg, rgu, gv -> uv",
        vxc[1],
        self.grids.weights,
        rho_1,
        self.ao_1,
        self.ao_0,
    )
    xc_v = xc_v + xc_v.T
    vjk = self.myhf.get_jk(self.mol, dm1, 1)

    fock_a = self.mat_hs @ (self.h1e + vjk[0] + xc_v) @ self.mat_hs
    _, mo = np.linalg.eigh(fock_a)
    mo = self.mat_hs @ mo
    dm1_old = dm1.copy()
    dm1 = 2 * mo[:, : self.nocc] @ mo[:, : self.nocc].T
    error = np.linalg.norm(dm1 - dm1_old)
    dm1 = self.hybrid(dm1, dm1_old)

self.logger.info(f"step: {step:<8} error of dm1, {error:.2e}\n")
self.logger.info(
    f"After scf: {np.array2string(dm1, precision=4, separator=',', suppress_small=True)}\n"
)

[ 43.03 149.17]
[ 68.08 315.35]


In [ ]:
plt.plot(train_set[:, 0], "r")
plt.plot(eval__set[:, 0], "b")
plt.plot(train_set[:, 1], "r--")
plt.plot(eval__set[:, 1], "b--")
plt.show()

In [4]:
import numpy as np
print(np.linspace(-0.25, 0.25, 11))
print(np.linspace(-0.225, 0.225, 10))
print(np.linspace(-0.5, -0.275, 10))
print(np.linspace(0.275, 0.5, 10))

print(np.linspace(-0.475, -0.3, 8))
print(np.linspace(-0.5, -0.275, 5))


[-0.25 -0.2  -0.15 -0.1  -0.05  0.    0.05  0.1   0.15  0.2   0.25]
[-0.225 -0.175 -0.125 -0.075 -0.025  0.025  0.075  0.125  0.175  0.225]
[-0.5   -0.475 -0.45  -0.425 -0.4   -0.375 -0.35  -0.325 -0.3   -0.275]
[0.275 0.3   0.325 0.35  0.375 0.4   0.425 0.45  0.475 0.5  ]
[-0.475 -0.45  -0.425 -0.4   -0.375 -0.35  -0.325 -0.3  ]
[-0.5     -0.44375 -0.3875  -0.33125 -0.275  ]


In [ ]:
0.4437 0.3875 0.3313